In [1]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint

In [2]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Osama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [4]:
train_classification_df.groupby("category").count()

,user_id
category,
art,191
entertainment,323
fashion,299
food,511
gaming,13
health and lifestyle,503
mom and children,149
sports,113
tech,346


In [5]:
username2_category["kod8net"]

'tech'

In [6]:
'''train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]




# Load the additional CSV file
additional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"
additional_data_df = pd.read_csv(additional_data_path)

# Extract and rename the relevant columns
additional_data_df = additional_data_df[['Unnamed: 0', 'influencerCategory']].rename(columns={
    'Unnamed: 0': 'user_id', 
    'influencerCategory': 'category'
})

additional_data_df = additional_data_df.dropna()



# Convert the 'category' column to string type and apply .lower(), handling any NaN or unexpected values
additional_data_df['category'] = additional_data_df['category'].astype(str).fillna('').apply(str.lower)

# Append the new data to the original train_classification_df
train_classification_df = pd.concat([train_classification_df, additional_data_df], ignore_index=True)

# Update the username2_category dictionary with the new data
username2_category.update(additional_data_df.set_index("user_id").to_dict()["category"])

# Check the updated data by viewing the first few rows
print(train_classification_df.head())

# Re-check the category distribution
train_classification_df.groupby("category").count()'''


'train_classification_df = pd.read_csv("/Users/Osama/Downloads/CS412PROJ/train-classification.csv",)\ntrain_classification_df = train_classification_df.rename(columns={\'Unnamed: 0\': \'user_id\', \'label\': \'category\'})\n\n# Unifying labels\ntrain_classification_df["category"] = train_classification_df["category"].apply(str.lower)\nusername2_category = train_classification_df.set_index("user_id").to_dict()["category"]\n\n\n\n\n# Load the additional CSV file\nadditional_data_path = "/Users/Osama/Downloads/CS412PROJ/annotated_users_CS412-2753ef4cf74e.csv"\nadditional_data_df = pd.read_csv(additional_data_path)\n\n# Extract and rename the relevant columns\nadditional_data_df = additional_data_df[[\'Unnamed: 0\', \'influencerCategory\']].rename(columns={\n    \'Unnamed: 0\': \'user_id\', \n    \'influencerCategory\': \'category\'\n})\n\nadditional_data_df = additional_data_df.dropna()\n\n\n\n# Convert the \'category\' column to string type and apply .lower(), handling any NaN or unexpec

In [7]:
train_data_path = "/Users/Osama/Downloads/CS412PROJ/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile

In [8]:
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.shape

(2741, 44)

In [9]:
test_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,beyazyakaliyiz,8634457436,Selam Beyaz Yakalı,Beyaz yakalıların dünyasına hoşgeldiniz 😀😀😀,Personal blog,None,1265,665,True,False,...,None,None,PERSONAL_BLOG,False,False,https://instagram.fist6-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,totalenergies_istasyonlari,7066643793,TotalEnergies İstasyonları,TotalEnergies İstasyonları resmi Instagram hes...,Energy Company,None,28025,4,True,False,...,None,None,ENERGY_COMPANY,False,False,https://instagram.fsaw2-1.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import emoji


def preprocess_text(text: str):
    

    # lower casing Turkish Text, Don't use str.lower :)
    text = text.casefold()

    #text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters and punctuation
    # HERE THE EMOJIS stuff are being removed, you may want to keep them :D
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove extra whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text


corpus = []

# to keep the label order
train_usernames = []

for username, posts in username2posts_train.items():
  train_usernames.append(username)

  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)


  # joining the posts of each user with a \n
  user_post_captions = "\n".join(cleaned_captions)
  corpus.append(user_post_captions)



#custom_stopwords = list(set(turkish_stopwords).union({
 #   'the', 'and', 'with', 'for', 'you', 'to', 'of', 'in', 'our', 'your', 'is', 'are','bir'
#}))
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=10000,min_df=10,ngram_range=(1, 3))

# fit the vectorizer
vectorizer.fit(corpus)

# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
feature_names = vectorizer.get_feature_names_out()

# Inspect the frequency of each word
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)

# Show the most frequent words (words in many posts)
print(df_tfidf.sum().sort_values(ascending=False).head(30))




test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

bir            192.469930
the            106.216595
and             65.958949
to              59.341887
of              59.223468
olsun           56.720778
yeni            46.531377
istanbul        46.082455
in              44.906322
olarak          43.515614
olan            41.210989
with            39.917915
kutlu           39.887376
kutlu olsun     39.310406
iyi             37.609202
güzel           37.208822
kadar           36.264804
you             35.986373
for             34.861254
özel            32.565787
bilgi           32.487925
devam           32.118460
var             31.813387
our             31.541669
kasım           31.359690
günü            31.187668
birlikte        30.766957
büyük           30.193571
ilk             29.995666
teşekkür        29.514784
dtype: float64


In [11]:
# Making sure everything is fine
assert y_train.count("NA") == 0


In [12]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['ab', 'abant', 'abd', ..., 'şıklık', 'şıklığı', 'şımartın'],
      dtype=object)

In [13]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.042156,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [14]:
df_tfidf.shape

(2741, 10000)

In [15]:
print("Shape of df_tfidf:", df_tfidf.shape)
print("Length of y_train:", len(y_train))

Shape of df_tfidf: (2741, 10000)
Length of y_train: 2741


In [16]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train,random_state=42)

In [17]:
x_train.shape

(2192, 10000)

In [18]:
x_val.shape

(549, 10000)

In [19]:
#@title Test Data
test_data_path = "/Users/Osama/Downloads/CS412PROJ/test-classification-round2.dat"

with open(test_data_path, "rt") as fh:
    for i, line in enumerate(fh):
        print(line.strip())
        if i >= 4:  # Stop after 5 lines
            break

print("*****")

test_unames = []
with open(test_data_path, "rt") as fh:
  for line in fh:
    test_unames.append(line.strip())

print(test_unames[:5])

schirogomme
takviyegiller
egeasalmetal_
expomedeurasia
izmirhim
*****
['schirogomme', 'takviyegiller', 'egeasalmetal_', 'expomedeurasia', 'izmirhim']


In [20]:
x_test = []

for uname in test_unames:
  try:
    index = test_usernames.index(uname)
    x_test.append(x_post_test[index].toarray()[0])
  except Exception as e:
    try:
      index = train_usernames.index(uname)
      x_test.append(x_post_train[index].toarray()[0])
    except Exception as e:
      print(uname)


#test_unames.remove("screenname")

In [21]:
df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
df_test.head(2)

,ab,abant,abd,abdulkadir,abdullah,abi,abiye,abone,about,about the,...,şöyle,şükran,şükranla,şükranlarımızı,şükür,şık,şık bir,şıklık,şıklığı,şımartın
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
def predict_like_count(username, current_post=None):
  def get_avg_like_count(posts:list):
    total = 0.
    for post in posts:
      if current_post is not None and post["id"] == current_post["id"]:
        continue

      like_count = post.get("like_count", 0)
      if like_count is None:
        like_count = 0
      total += like_count

    if len(posts) == 0:
      return 0.

    return total / len(posts)

  if username in username2posts_train:
    return get_avg_like_count(username2posts_train[username])
  elif username in username2posts_test:
    return get_avg_like_count(username2posts_test[username])
  else:
    print(f"No data available for {username}")
    return -1

In [23]:
def log_mse_like_counts(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [24]:
#@title Train Dataset evaluation

y_like_count_train_true = []
y_like_count_train_pred = []
for uname, posts in username2posts_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train= {log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train= 1.2271047744059362


In [25]:
#@title Test Dataset

path = "/Users/Osama/Downloads/CS412PROJ/test-regression-round1.jsonl"
output_path = "/Users/Osama/Downloads/CS412PROJ/testy-regression-round1.jsonl"

to_predict_like_counts_usernames = []
output_list = []
with open(path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    # let's predict
    pred_val = predict_like_count(sample["username"])
    sample["like_count"] = int(pred_val)
    output_list.append(sample)

with open(output_path, "wt") as of:
  json.dump(output_list, of)




In [26]:
pprint(output_list[:3])

[{'caption': 'KOZA 2023 2.si Damla’nın koleksiyonu, Latincede ‘Memento Mori’ '
             'olarak bilinen ‘ölümlü olduğunu hatırla’ anlamındaki ifadeden '
             'esinleniyor. Koleksiyon, hayatın ve ölümün, para, işçi, kral ve '
             'kraliçe kavramları üzerinden yaratıcı görünümlerle bir araya '
             'getirilmesini amaçlıyor. Ölüm sembollerinden esinlenen desenler '
             'kullanan Damla, “kağıt parçasından ibaret olmak” kavramını '
             'vurguluyor. Koleksiyon, yaşamın ve ölümün aynı anda ifade '
             'edilmesini hedefliyor; kırmızı ve mavi ışıklarla veya '
             'gözlüklerle görülen hologram efekti kullanılarak bu konsept '
             'sahneye taşınıyor. Kırmızı renk ölümü, mavi ise yaşamı '
             'simgeliyor. Koleksiyon, ofis giyimlerinden esinlenerek '
             'kravatlar, gömlekler ve evrak çantaları içeriyor. Klasik sivri '
             'burun çizmelerin üzerine spor ayakkabıların üst yüzeyi '
             'yerle

In [ ]:
'''from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor


# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Initialize the regressor
rf = RandomForestRegressor(random_state=42)

# Perform grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Best parameters and best model
print("Best Parameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
'''

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


ValueError: 
All the 1620 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
269 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\bas

In [66]:
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]

# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

# Train an MLP Regressor
xgb_regressor = XGBRegressor(
    n_estimators=100,   # Number of trees
    learning_rate=0.1,  # Step size for updating weights
    max_depth=6,        # Depth of each tree
    random_state=42
)

# Train the model
xgb_regressor.fit(x_train, y_train)

y_val = np.maximum(y_val, 0)



# Predict on validation set
y_pred_val = xgb_regressor.predict(x_val)
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
print(f"MSE on validation set: {mse}")

# Predict on test set
y_test_pred = xgb_regressor.predict(x_post_test)


# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})
print(test_predictions.head())


MSE on validation set: 2819676604.812782
                     username  predicted_like_count
0              beyazyakaliyiz          10770.258789
1  totalenergies_istasyonlari            453.646881
2                 konforyatak            370.162567
3                    ht_kulup            484.007294
4                   ajansspor           2237.495117


In [ ]:
import statsmodels.api as sm
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]

# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

lgbm_regressor = LGBMRegressor(
    n_estimators=1000,
    learning_rate=0.2,
    max_depth=30,
    min_data_in_leaf=30,
    feature_fraction=0.8,
    bagging_fraction=0.8,
    objective="poisson",  # Poisson objective for count data
    random_state=42
)

# Train the model
lgbm_regressor.fit(x_train, y_train)
'''
# Predict on validation set
y_pred_val = lgbm_regressor.predict(x_val)

# Ensure non-negative predictions
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
print(f"MSE on validation set: {mse}")

# Predict on test set
y_test_pred = lgbm_regressor.predict(x_post_test)

# Ensure non-negative predictions
y_test_pred = np.maximum(y_test_pred, 0)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})

print(test_predictions.head())'''

C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.080365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 210149
[LightGBM] [Info] Number of data points in the train set: 2192, number of us

In [66]:
# Predict on validation set
y_pred_val = lgbm_regressor.predict(x_val)

# Ensure predictions are non-negative integers
y_pred_val = np.maximum(np.round(y_pred_val), 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
print(f"MSE on validation set: {mse}")

# Predict on test set
y_test_pred = lgbm_regressor.predict(x_post_test)

# Ensure predictions are non-negative integers
y_test_pred = np.maximum(np.round(y_test_pred), 0)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred.astype(int)  # Explicitly convert to integer
})

print(test_predictions.head())


[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
MSE on validation set: 3061672686.6953554
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8
                     username  predicted_like_count
0              beyazyakaliyiz                  9599
1  totalenergies_istasyonlari                    62
2                 konforyatak                    24
3                    ht_

In [67]:
print(test_predictions.head(50))

                      username  predicted_like_count
0               beyazyakaliyiz                  9599
1   totalenergies_istasyonlari                    62
2                  konforyatak                    24
3                     ht_kulup                   169
4                    ajansspor                   345
5         yusufelibelediyesi08                    73
6                     4bros.tr                   105
7              groundy.kadikoy                   455
8                drtubagunebak                   308
9                   nihatcan11                    63
10               mustafaaakcay                   267
11                  mtmofamily                  9031
12                 bonitoperde                   133
13     balkanlardangelenlezzet                    15
14      pedagoganne__gulozturk                   517
15                      iktm34                   292
16                    yudumyag                   246
17      bulentozdemir.edebiyat                

In [68]:
if np.isnan(y_val).any() or np.isnan(y_pred_val).any():
    print("NaN values found in y_val or y_pred_val!")


In [69]:
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values gracefully

# Compute the absolute log errors
abs_log_errors = np.abs(log_errors)

# Sum of log errors
sum_log_errors = np.sum(abs_log_errors)

print(f"Sum of Log Errors: {sum_log_errors}")

Sum of Log Errors: 728.0208173955898


In [70]:
test_pred = lgbm_regressor.predict(df_test)

output = dict()
for index, uname in enumerate(test_unames):
  output[uname] = test_pred[index]

[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] bagging_fraction is set=0.8, subsample=1.0 will be ignored. Current value: bagging_fraction=0.8


In [71]:
with open("prediction-regression-round2.json", "w") as of:
  json.dump(output, of, indent=4)

In [140]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.05, 0.1, 0.15],
    'max_depth': [5, 10, -1],
    'min_data_in_leaf': [10, 20, 30],
    'feature_fraction': [0.8, 0.9],
    'bagging_fraction': [0.8, 0.9]
}

from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(estimator=lgbm_regressor, param_grid=param_grid, cv=3, n_jobs=-1, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)

print("Best parameters found: ", grid_search.best_params_)


KeyboardInterrupt: 

In [78]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Prepare the target variable: Average like count per user
user_avg_likes = {}
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    user_avg_likes[username] = avg_likes

# Create the target array
y_like_counts = [user_avg_likes[uname] for uname in train_usernames]
y_like_counts = [0 if np.isnan(val) else val for val in y_like_counts]
# Train-test split
x_train, x_val, y_train, y_val = train_test_split(
    x_post_train, y_like_counts, test_size=0.2, random_state=42
)

xgb_regressor = XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=6,
    objective='reg:squarederror',  # Default; can change to 'reg:poisson' for count data
    random_state=42
)

# Train the model
xgb_regressor.fit(x_train, y_train)

# Predict on validation set
y_pred_val = xgb_regressor.predict(x_val)

# Post-process: Clamp negative predictions to 0
y_pred_val = np.maximum(y_pred_val, 0)

# Evaluate the model
mse = mean_squared_error(y_val, y_pred_val)
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values
abs_log_errors = np.abs(log_errors)
sum_log_errors = np.sum(abs_log_errors)

print(f"MSE on validation set: {mse}")
print(f"Sum of Log Errors: {sum_log_errors}")

# Predict on test set
y_test_pred = xgb_regressor.predict(x_post_test)

# Post-process: Clamp negative predictions to 0
y_test_pred = np.maximum(y_test_pred, 0)

# Create a DataFrame for test predictions
test_predictions = pd.DataFrame({
    "username": test_usernames,
    "predicted_like_count": y_test_pred
})

print(test_predictions.head())

# Save test predictions to a CSV file
#test_predictions.to_csv("test_predictions.csv", index=False)


C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Osama\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\_core\_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


MSE on validation set: 2819676604.812782
Sum of Log Errors: 1278.0898952402472
                     username  predicted_like_count
0              beyazyakaliyiz          10770.258789
1  totalenergies_istasyonlari            453.646881
2                 konforyatak            370.162567
3                    ht_kulup            484.007294
4                   ajansspor           2237.495117


In [79]:
log_errors = np.log1p(y_val) - np.log1p(y_pred_val)  # Use np.log1p to handle zero values gracefully

# Compute the absolute log errors
abs_log_errors = np.abs(log_errors)

# Sum of log errors
sum_log_errors = np.sum(abs_log_errors)

print(f"Sum of Log Errors: {sum_log_errors}")

Sum of Log Errors: 1278.0898952402472


In [36]:
# Check for NaN in user_avg_likes
for username, posts in username2posts_train.items():
    avg_likes = np.mean([post.get("like_count", 0) or 0 for post in posts])
    if np.isnan(avg_likes):
        print(f"NaN detected for user: {username}, posts: {posts}")
    user_avg_likes[username] = avg_likes


NameError: name 'user_avg_likes' is not defined

In [28]:
'''# Calculate log errors
log_errors = np.abs(y_val_reg - y_val_pred_reg)

# Sum of log errors
sum_log_errors = np.sum(log_errors)

print("Sum of Log Errors:", sum_log_errors)
'''

NameError: name 'y_val_pred_reg' is not defined